In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy
import datetime

### Read Data

In [7]:
# modify the read_csv path to personal computer
data = pd.read_csv("~/Columbia/Personalization Theory/lastfm-dataset-1K/userid-timestamp-artid-artname-traid-traname.tsv", 
                   delimiter="\t", header=None, 
                   names = ["userid","timestamp","musicbrainz-artist-id",
                            "artist-name","musicbrainz-track-id","track-name"])


In [8]:
user_profiles = pd.read_csv("~/Columbia/Personalization Theory/lastfm-dataset-1K/userid-profile.tsv", delimiter = "\t")
user_profiles.columns = ["userid", "gender","age","country","registered"]
user_profiles.drop(["country", "registered"], axis=1, inplace=True)

### Make timestamps, days of week, hours. Sort by time and users

In [9]:
data = data.rename(columns={0:'userid', 1:'timestamp', 2:'artistid', 3:'artistname', 4:'trackid', 5:'trackname'})
data['timestamp'] = pd.to_datetime(data['timestamp'])

In [10]:
data['weekday'] = data['timestamp'].dt.weekday_name.astype('category',
                                                           categories=["Sunday","Monday","Tuesday","Wednesday",
                                                                       "Thursday","Friday","Saturday"],
                                                           ordered=True)
data['hour'] = data['timestamp'].dt.hour


In [11]:
data = data.sort_values(['userid', 'timestamp'], ascending=[True, True])

### Time of day (morning, noon etc)

In [12]:
data['weekend'] = 0 
data.loc[data['weekday'].isin(['Saturday', 'Sunday']), 'weekend'] = 1

# data['timestamp'] = pd.to_datetime(data['timestamp'])
# data_user['hour'] = data_user['timestamp'].dt.hour

data['daytime'] = 0
data.loc[(data['hour'] >= 0) & (data['hour'] <= 3), 'daytime'] = 0
data.loc[(data['hour'] > 3) & (data['hour'] <= 5), 'daytime'] = 1
data.loc[(data['hour'] > 5) & (data['hour'] <= 11), 'daytime'] = 2
data.loc[(data['hour'] > 11) & (data['hour'] <= 19), 'daytime'] = 3
data.loc[(data['hour'] > 19) & (data['hour'] <= 23), 'daytime'] = 4

### count of how many times an SONG was listened to in given day and time

In [13]:
data['counted'] = 1

#data_user_counted = data.groupby(['userid','track-name', 'daytime', 'weekend'])['counted'].count().reset_index()
#data_user_counted.columns = ['userid','track-name', 'daytime', 'weekend', 'track-weekday-daytime-count']

data['track-total-count'] = data.groupby(['userid','track-name']).cumcount() + 1
data['track-weekday-daytime-count'] = data.groupby(['userid','track-name', 'daytime', 'weekend']).cumcount() + 1

#data = pd.merge(data, data_user_counted, on = ['userid','track-name', 'daytime', 'weekend'], how = 'left')

### Determine how long a song was played and when was it last played

In [14]:
data['songlength'] = data.groupby('userid')['timestamp'].diff()
data['last-seen-song'] = data.groupby(['track-name'])['timestamp'].diff()
data['last-seen-song'] = data['last-seen-song'].dt.total_seconds() / (24 * 60 * 60)

### Quarters of the year

In [15]:
data['month'] = data['timestamp'].dt.month
data['quarter'] = 1
data.loc[(data['month'] > 3) & (data['month'] <= 6), 'quarter'] = 2
data.loc[(data['month'] > 6) & (data['month'] <= 9), 'quarter'] = 3
data.loc[(data['month'] > 9) & (data['month'] <= 12), 'quarter'] = 4

## SKIPS variable - The independent variable

In [16]:
data['skipped'] = (data.songlength  < datetime.timedelta(minutes=1)).astype('int')

### count of how many times an ARTIST was listened to in given day and time

In [17]:
#data_user_counted = data.groupby(['userid','artist-name', 'daytime', 'weekend'])['counted'].count().reset_index()
#data_user_counted.columns = ['userid','artist-name', 'daytime', 'weekend', 'artist-weekday-daytime-count']

data['artist-total-count'] = data.groupby(['userid','artist-name']).cumcount() + 1
data['artist-weekday-daytime-count'] = data.groupby(['userid','artist-name', 'daytime', 'weekend']).cumcount() + 1

#data = pd.merge(data, data_user_counted, on = ['userid','artist-name', 'daytime', 'weekend'], how = 'left')

### Determine when an artist was last played

In [18]:
data['last-seen-artist'] = data.groupby(['artist-name'])['timestamp'].diff()
data['last-seen-artist'] = data['last-seen-artist'].dt.total_seconds() / (24 * 60 * 60)

### Merge user metadata

In [19]:
data = pd.merge(data, user_profiles, on = ['userid'], how = 'left')

In [20]:
data.drop(["musicbrainz-artist-id", "musicbrainz-track-id","counted"], axis=1, inplace=True)

### Convert days into numerical values (Mon = 0, Sun = 6)

In [21]:
data['weekday'] = data['timestamp'].dt.dayofweek

### Pick top 1000 song for each user

In [22]:
import numpy as np

In [23]:
songs = data.groupby(['userid', 'track-name', 'artist-name']).size().reset_index(name="count")


In [24]:
top1000 = songs.groupby(['userid'])['track-name','artist-name','count'].apply(lambda grp: grp.nlargest(1000, 'count')).reset_index()


In [25]:
top1000.head()

,userid,level_1,track-name,artist-name,count
0,user_000001,1654,Music,Cornelius,70
1,user_000001,998,Gum,Cornelius,63
2,user_000001,1525,Mario Basanov & Vidis ‘Test’,Gilles Peterson,52
3,user_000001,441,Child Song,The Cinematic Orchestra,45
4,user_000001,1051,Hibari,坂本龍一,42


In [26]:
top1000.drop(["level_1", "count"], axis=1, inplace=True)

In [27]:
data_subset = pd.merge(top1000, data, on = ['userid','track-name', 'artist-name'], how = 'left')

In [28]:
print(np.shape(data))
print(np.shape(data_subset))

(19098862, 20)
(10543035, 20)


In [29]:
data_subset = data_subset.sort_values(['userid', 'timestamp'], ascending=[True, True])

In [30]:
data_subset.head(5)

,userid,track-name,artist-name,timestamp,weekday,hour,weekend,daytime,track-total-count,track-weekday-daytime-count,songlength,last-seen-song,month,quarter,skipped,artist-total-count,artist-weekday-daytime-count,last-seen-artist,gender,age
8512,user_000001,The Launching Of Big Face,Plaid & Bob Jaroc,2006-08-13 13:59:20,6,13,1,3,1,1,NaT,NaN,8,3,0,1,1,NaN,m,NaN
7890,user_000001,Zn Zero,Plaid & Bob Jaroc,2006-08-13 14:03:29,6,14,1,3,1,1,00:04:09,NaN,8,3,0,2,2,0.002882,m,NaN
3782,user_000001,The Return Of Super Barrio - End Credits,Plaid & Bob Jaroc,2006-08-13 14:10:43,6,14,1,3,1,1,00:07:14,NaN,8,3,0,3,3,0.005023,m,NaN
3101,user_000001,Dayvan Cowboy,Boards Of Canada,2006-08-13 15:44:17,6,15,1,3,1,1,00:04:04,NaN,8,3,0,1,1,NaN,m,NaN
5295,user_000001,A Moment Of Clarity,Boards Of Canada,2006-08-13 16:46:52,6,16,1,3,1,1,00:00:40,NaN,8,3,1,3,3,0.000463,m,NaN


### Write to CSV

In [31]:
data_subset.to_csv("data_engineered_features.csv", index=False)

### Also do just top 500?

In [32]:
top500 = songs.groupby(['userid'])['track-name','artist-name','count'].apply(lambda grp: grp.nlargest(500, 'count')).reset_index()

In [33]:
top500.drop(["level_1", "count"], axis=1, inplace=True)
data_subset = pd.merge(top500, data, on = ['userid','track-name', 'artist-name'], how = 'left')
data_subset = data_subset.sort_values(['userid', 'timestamp'], ascending=[True, True])
data_subset.to_csv("data_engineered_features500.csv", index=False)

In [34]:
np.shape(top500)

(468483, 3)